In [11]:
#QRM gate counts
#June 27th 2023
import math
import numpy as np
from qrm_counts import *
r, m = 7, 10

print(naive_CX_count(r, m))
print(std_CX_count(r, m))
print(rec_CX_count(r, m))

param_list = [(1, 3), (2, 4), (3, 4), (2, 5), (3, 5), (4, 5), (5, 5), (3, 6), (4, 6), (5, 6), (3, 7), (4, 7), (5, 7), (6, 7), (4, 8), (5, 8), (6, 8), (7, 8), (8, 10), (9, 10)]

print(rec_CX_count_assym(2, 5, 2, 5))

for params in param_list:
    r, m = params
    print('\n', r, m)
    print(naive_CX_count(r, m))
    print(std_CX_count(r, m))
    print(rec_CX_count(r, m))

57880
20664
5096
66

 1 3
16
12
10

 2 4
61
53
30

 3 4
65
29
32

 2 5
176
112
66

 3 5
206
150
78

 4 5
211
61
80

 5 5
211
0
80

 3 6
614
470
176

 4 6
659
367
190

 5 6
665
125
192

 3 7
1744
960
372

 4 7
1989
1389
430

 5 7
2052
840
446

 6 7
2059
253
448

 4 8
5821
3933
930

 5 8
6213
3605
1004

 6 8
6297
1857
1022

 7 8
6305
509
1024

 8 10
58015
8627
5118

 9 10
58025
2045
5120


In [5]:
rec_CX_count_assym()

65

In [ ]:
print(rec_CX_count_assym(2, 5, 2, 5))

In [5]:
from qrm_counts import rec_CX_count_assym, Urm_CX_count, Urr_CX_count
r, m = (2, 4)
print(rec_CX_count_assym(r, m, r, m))
print(Urm_CX_count(r, r), Urr_CX_count(r))

25
4 4


In [3]:
#CSS encoders
import tequila
from qrm_circuits import QRM_std_circuit, QRM_rec_circuit
from qrm_utils import *
from qrm_circuits import QRM_std_circuit
from qrm_matrices import *

#need to generalize stuff
r, m = 2, 4
circuit, info = QRM_rec_circuit(r, m, partitions=[1, [1], [1]])
std_circuit, info2 = QRM_std_circuit(r, m)
qiskit_cir = get_qiskit_circuit(circuit)
print(info, qiskit_cir, circuit.depth, len(circuit.gates))
check_if_same_circuit(circuit, std_circuit)

([3, 5, 6, 9, 10, 12], [1, 2, 4, 0, 8])       ┌───┐                                                                 
 q_0: ┤ H ├─────────────────────────────────────■───────────────────■────■──
      ├───┤                                     │                   │  ┌─┴─┐
 q_1: ┤ H ├─────────────────■───────────────────┼────■─────────■────┼──┤ X ├
      ├───┤                 │                   │    │         │  ┌─┴─┐└───┘
 q_2: ┤ H ├─────────────────┼────■──────────────┼────┼────■────┼──┤ X ├──■──
      └───┘                 │    │              │    │    │  ┌─┴─┐└───┘┌─┴─┐
 q_3: ───────■──────────────┼────┼────■─────────┼────┼────┼──┤ X ├─────┤ X ├
      ┌───┐  │              │    │    │       ┌─┴─┐  │    │  └───┘     └───┘
 q_4: ┤ H ├──┼──────────────┼────┼────┼────■──┤ X ├──┼────┼─────────■────■──
      └───┘  │              │    │    │    │  └───┘┌─┴─┐  │         │  ┌─┴─┐
 q_5: ───────┼────■─────────┼────┼────┼────┼───────┤ X ├──┼────■────┼──┤ X ├
             │    │         │    │  

True

In [ ]:
def QRM0_rec_circuit(r, m, partitions = [], qubit_list = None, only_cnots = False):
    U = tequila.QCircuit()
    msg_q = []
    ent_q = []
    if m == 0:
        return U, (qubit_list, [])
    if qubit_list == None:
        qubit_list = list(range(2**m))
    
    if partitions != []:
        ql_1, ql_2 = apply_qubit_partition(partitions[0], qubit_list)
        if len(partitions) > 1:
            p1 = partitions[1]
            if len(partitions) > 2:
                p2 = partitions[2]
            else:
                p2 = []
        else:
            p1, p2 = [], []
    else:
        ql_1, ql_2 = qubit_list[:2**(m-1)], qubit_list[2**(m-1):]
        p1, p2 = [], []
    
    if r == m:
        for a, b in zip(ql_1, ql_2):
            U += tequila.gates.CX(control=a, target=b)
        U1, info_1 = QRM_rec_circuit(r-1, m-1, partitions=p1, qubit_list=ql_1)
        U2, info_2 = QRM_rec_circuit(r-1, m-1, partitions=p2, qubit_list=ql_2)
        U = U + U1 + U2
        return U, (qubit_list, info_1[1] + info_2[1])
    
    U1, info_1 = QRM_rec_circuit(r, m-1, partitions=p1, qubit_list=ql_1)
    U2, info_2 = QRM_rec_circuit(r, m-1, partitions=p2, qubit_list=ql_2)
    if r > m-r-1:
        #add message ents
        Gm = get_QRM_generators_r1r2(m-r, r, m-1)
        lbs = [leading_bit_index(row) for row in Gm]
        qm1 = [ql_1[i] for i in lbs]
        qm2 = [ql_2[i] for i in lbs]
        for a, b in zip(qm1, qm2):
            U += tequila.gates.CX(control=a, target=b)
        msg_q += qm1
    
    Gent = get_QRM_generators_r1r2(m-r-1, m-r-1, m-1)
    lbs = [leading_bit_index(row) for row in Gent]
    qent1 = [ql_1[i] for i in lbs]
    qent2 = [ql_2[i] for i in lbs]
    if not only_cnots:
        U += tequila.gates.H(target=qent1)
    for a, b in zip(qent1, qent2):
        U += tequila.gates.CX(control=a, target=b)

    U += U1
    U += U2

    ent_q += qent1 + info_1[1] + info_2[1]
    msg_q = [leading_bit_index(row) for row in get_QRM_generators_r1r2(m-r, r, m)] #This is a placeholder. Need to figure out message bit pass recursively, should be msg_q + something
    return U, (msg_q, ent_q)

In [1]:
import tequila
from qrm_circuits import QRM_rec_circuit, QRM_std_circuit
only_cnots = False
circuit, info1 = QRM_rec_circuit(3, 4, only_cnots=only_cnots)
circuit_std, info2 = QRM_std_circuit(3, 4, only_cnots=only_cnots, transform_rows=False)
msg = tequila.QCircuit()

msg += tequila.gates.H(target=info1[0][0])
msg += tequila.gates.CX(target=info1[0][1], control=info1[0][0])
print(len(circuit_std.gates), len(circuit.gates))
from qrm_utils import check_if_same_circuit

check_if_same_circuit(msg + circuit, msg + circuit_std)

66 33


True

In [4]:
#generate check matrix and find the diagonalizing circuit
from qrm_matrices import *
r, m = (2, 3)
G = Grm(r, m)
H = Hrm(r, m)
#HCSS = 

import tequila
from tequila.grouping.binary_rep import BinaryPauliString, BinaryHamiltonian

H = Hqrm(1, 3)

bps = [BinaryPauliString(binary_vector=bv, coeff=1.0) for bv in H]
BH = BinaryHamiltonian(binary_terms=bps)

stuff = BH.get_qubit_wise()

ImportError: attempted relative import with no known parent package